In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from matplotlib import animation
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout)

# Aprendizaje Supervisado

Esquema donde se busca aprender un mapeo o función 

$$
f_{\theta}: \mathcal{X} \rightarrow \mathcal{Y},
$$

donde $\mathcal{X}$ es el dominio de nuestros datos (entrada) e $\mathcal{Y}$ es un objetivo (salida)


Entrenamos nuestro modelo a partir de un conjunto de $N$ ejemplos:

$$
\{(x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N)\},
$$

donde cada ejemplo es una tupla formada de datos $x_i \in \mathcal{X}$ y objetivo $y_i \in \mathcal{Y}$

Si la variable objetivo es
- continua: hablamos de un problema de regresión o aproximación de funciones
- categórica: hablamos de un problema de clasificación

La naturaleza de los datos  depende del problema

Lo más común es que los datos $x_i$ se estructuren como arreglos de $M$ componentes

A los componentes los llamamos atributos o *features*


### Aprendizaje

El vector $\theta$ corresponde a los **parámetros** del modelo

> Aprender o ajustar el modelo corresponde a encontrar el valor "óptimo" de $\theta$ 

Usamos una **función de pérdida/costo** $L(\theta)$ para medir el error de nuestro modelo

> **Minimizamos** la función de costo para encontrar el mejor $\theta$


Cuando hablamos de $\theta$ "óptimo" lo decimos en el sentido de una función de costo particular


### Optimización

Queremos resolver el siguiente problema
$$
\min_\theta L(\theta)
$$

Una opción es evaluar $L()$ en todo el espacio de posibles $\theta$: Fuerza Bruta

Pero en general esto no es computacionalmente posible

Podemos usar **técnicas de optimización** para encontrar el mejor $\theta$

Si $L(\theta)$ es continua y derivable podemos escribir

$$
\nabla_\theta L(\theta) = \vec 0
$$

e intentar despejar $\theta~$

## Regresión lineal

Modelo para aprender una mapeo entre una o más variables continuas (atributos) hacia una variable continua (objetivo)

En un esquema con $M$ atributos y $N$ ejemplos tenemos

$$
y_i = f_\theta(\vec x_i) = \vec w^T \vec x_i + b = \sum_{j=1}^M w_j x_{ij} + b, 
$$

donde $f_\theta$ es un modelo parámetrico (hiperplano) con $M+1$ parámetros 

$$
\theta= \begin{pmatrix} b \\ w_1 \\ w_2 \\ \vdots \\ w_M \end{pmatrix}
$$

También podemos escribir el sistema matricialmente como

$$
Y = X \theta
$$

donde $Y= \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\y_N\end{pmatrix} \in \mathbb{R}^N$, $X = \begin{pmatrix} 1 & x_{11} & x_{12}& \ldots& x_{1M} \\ \vdots & \vdots & \vdots& \ddots& \vdots \\ 1 & x_{N1} & x_{N2}& \ldots& x_{NM} \\ \end{pmatrix} \in \mathbb{R}^{N\times M}$ y $\theta \in \mathbb{R}^M$

Una función de costo razonable es

$$
L(\theta) = \frac{1}{2} (Y - X\theta)^T (Y - X\theta)
$$

que corresponde al cuadrado de los errores 

> Esta función de costo se conoce como Error Cuadrático Medio

Luego si derivamos e igualamos a cero obtenemos

$$
\nabla_\theta L(\theta) = -X^T(Y-X\theta) = 0,
$$

y despejando

$$
\hat \theta = (X^T X)^{-1} X^T Y
$$

siempre y cuando podamos invertir $X^T X$

> Esto se conoce como solución de mínimos cuadrados



### Regresión lineal con funciones base 

Podemos generalizar el regresor lineal aplicando transformaciones a $X$

Por ejemplo una regresión polinomial de grado $M$ sería

$$
y_i = f_\theta(x_i) = \sum_{j=1}^M w_j x_{i}^j + b, 
$$

y su solución sería

$$
\hat \theta = (\Phi^T \Phi)^{-1} \Phi^T Y,
$$

donde $\Phi = \begin{pmatrix} 1 & x_1 & x_1^2& \ldots& x_1^M \\ \vdots & \vdots & \vdots& \ddots& \vdots \\ 1 & x_N & x_N^2& \ldots& x_N^M \\ \end{pmatrix}$

### Ejemplo

Considere el siguiente conjunto de datos $X,Y$ sintéticos con $Y$ continuo separados en conjunto de entrenamiento y validación

Usaremos $2/3$ de la data para entrenar y $1/3$ para validar

La función anónima `poly_basis` recibe $X$ y retorna $\Phi$

In [ ]:
poly_basis = lambda x, M : np.vstack([x**k for k in range(M)]).T
theta_real = np.array([10, -2, -0.3, 0.1])

x = np.linspace(-5, 6, num=21); 
X = poly_basis(x, len(theta_real))
y = np.dot(X, theta_real)

rseed, sigma = 0, 1.
np.random.seed(rseed);
Y = y + sigma*np.random.randn(len(x))
P = np.random.permutation(len(x))
train_idx, valid_idx = P[:2*len(x)//3], P[2*len(x)//3:]

La función `np.linalg.lstsq` recibe $\Phi$ e $Y$ y retorna el resultado de mínimos cuadrados $\hat \theta$

El siguiente ejemplo estudia el resultado de la regresión usando distintos valores de $M$, es decir el grado del polinomio

- $M=0 \rightarrow$  constante
- $M=1 \rightarrow$  recta
- $M=2 \rightarrow$  parábola
- ...

En este ejemplo
- El "modelo real" es el que se usó para generar los datos. Normalmente no está disponible. 
- El "modelo aprendido" es el que ajustamos con la regresión polinomial. Idealmente queremos que este modelo se parezca al "modelo real"

Modifica $M$ y analiza los resultados del ajuste

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

def update_plot(ax, M):
    ax.cla();
    Phi = poly_basis(x, M)
    theta_hat = np.linalg.lstsq(Phi[train_idx, :], Y[train_idx], rcond=None)[0]
    ax.plot(x, y, 'g-', linewidth=2, label='Modelo real', alpha=0.6, zorder=-100)
    ax.scatter(x[train_idx], Y[train_idx], s=50, label='Entrenamiento')
    ax.scatter(x[valid_idx], Y[valid_idx], s=50, label='Validación')
    ax.vlines(x[train_idx], np.dot(Phi[train_idx, :], theta_hat), Y[train_idx])  
    ax.vlines(x[valid_idx], np.dot(Phi[valid_idx, :], theta_hat), Y[valid_idx])     
    x_plot = np.linspace(-5, 6, num=100);
    ax.plot(x_plot, np.dot(poly_basis(x_plot, M), theta_hat), 'k-', linewidth=2, label='Modelo aprendido')
    ax.set_ylim([-5, 15]); plt.legend()
    
widgets.interact(update_plot, 
                 ax=widgets.fixed(ax), 
                 M=IntSlider_nice(description='M: grado del polinomio', min=1, max=11));

# Complejidad y Sobreajuste

En el ejemplo anterior vimos que se puede obtener modelos más flexibles si aumentamos el grado del polinomio 

> Aumentar la cantidad de parámetros (grados de libertad) hace al modelo más flexible y más complejo

Si la flexibilidad es excesiva aproximamos los datos con cero error

Esto no es bueno ya que estamos aprendiendo "de memoria" los datos y ajustandonos al ruido

> Sobreajuste: Aprender perfectamente los datos usados para entrenar

El modelo sobreajustado predice muy mal los datos "que no ha visto"

> El sobreajuste es inversamente proporcional a la capacidad de generalización

Por esta razón usamos conjuntos de validación



Figura: https://www.d2l.ai/chapter_multilayer-perceptrons/underfit-overfit.html

# Representatividad y Validación

En Machine Learning el primer paso para entrenar nuestro modelo es obtener datos 

Es crítico que los datos que utilizemos **representen** adecuadamente el problema que queremos resolver

Sea un espacio de datos (circulo negro) y muestras (puntos azules), ¿que puede decir de los siguientes casos?

In [ ]:
from matplotlib.patches import Circle
fig, ax = plt.subplots(1, 2, figsize=(7, 3))
np.random.seed(19)
for ax_ in ax:
    ax_.axis('off') 
    ax_.set_xlim([-3, 3])
    ax_.set_ylim([-3, 3])
r = 2.5*np.random.rand(100); t = np.random.rand(100);
ax[0].scatter(r*np.cos(2.0*np.pi*t), r*np.sin(2.0*np.pi*t))
p = Circle((0, 0), 3, fill=False, ec='k')
ax[0].add_artist(p)
r = 2.5*np.random.rand(100); t = np.random.rand(100);
ax[1].scatter(r*np.cos(t), r*np.sin(t))
p = Circle((0, 0), 3, fill=False, ec='k')
ax[1].add_artist(p); 

Siempre que podamos controlar el proceso de muestreo debemos poner atención a evitar **sesgos**

Asumiendo que nuestro dataset es representativo el siguiente paso es **entrenar**

Para combatir el sobreajuste podemos usar **estrategias de validación**

Consisten en separar el conjunto en dos o más subconjuntos
- Holdout: Entrenamiento/Validación/Prueba
- K-fold cross-validation y Leave one-out (N-fold) cross-validation
- Versiones estratificadas/balanceadas de las anteriores

Para que nuestro conjuntos de entrenamiento y validación sigan siendo representativos del total los **seleccionamos aleatoriamente**

Medimos $L(\theta)$ en entrenamiento y validación

> Optimizamos nuestro modelo minimizando el error de entrenamiento

> Seleccionamos los parámetros e hiper-parámetros que dan mínimo error de validación

> Comparamos distintas familias de modelos  con el error de prueba



### Ejemplo

Apliquemos esto al ejemplo de regresión polinomial

¿Cómo varía el error de entrenamiento y validación con $M$?

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
M_values = np.arange(1, 11)
mse = np.zeros(shape=(len(M_values), 2))
for i, M in enumerate(M_values):
    Phi = poly_basis(x, M)
    theta_hat = np.linalg.lstsq(Phi[train_idx, :], Y[train_idx], rcond=None)[0]
    mse[i, 0] = np.mean(np.power(Y[train_idx] - np.dot(Phi[train_idx, :], theta_hat), 2))
    mse[i, 1] = np.mean(np.power(Y[valid_idx] - np.dot(Phi[valid_idx, :], theta_hat), 2))
ax.plot(M_values, mse[:, 0], label='Entrenamiento')
ax.plot(M_values, mse[:, 1], label='Validación')
plt.legend()
ax.set_ylim([1e-4, 1e+4])
ax.set_yscale('log')
ax.set_xlabel('Grado del polinomio')
ax.set_ylabel('Loss');

# El punto representa el mejor M de validación
idx_best = np.argmin(mse[:, 1])
ax.scatter(M_values[idx_best], mse[idx_best, 1], c='k', s=100);

En resumen

- Bajo error de entrenamiento y de validación: **Ideal**

- Bajo error de entrenamiento y alto error de validación: **Modelo sobreajustado**

- Alto error de entrenamiento y de validación: Considera otro modelo y/o revisa tu código


## Neurona artificial o regresor logístico

Es un modelo para aprender un mapeo desde una o más variables continuas (atributos) hacia una variable binaria (objetivo)

Definamos primero la función sigmoide o logística

$$
\mathcal{S}(z) = \frac{1}{1+\exp(-z)}
$$

Esta función mapea un variable real al espacio $[0, 1]$

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), tight_layout=True)

sigmoid = lambda z : 1./(1+np.exp(-z))
z = np.linspace(-6, 6, num=1000)
ax.plot(z, sigmoid(z))
dot = ax.plot(0, sigmoid(0), 'ko')

def update_plot(z):
    dot[0].set_xdata(z)
    dot[0].set_ydata(sigmoid(z))
    
widgets.interact(update_plot, z=FloatSlider_nice(min=-5, max=5, value=0));

Luego definamos la siguiente transformación lineal de los datos de entrada

$$
z_i = \theta_0 + \sum_{j=1}^M \theta_j x_{ij}
$$

El modelo de regresión logística aplica la función sigmoide a la transformación lineal

$$
f_\theta(\vec x_i) = \mathcal{S} \left(\theta_0 + \sum_{j=1}^M \theta_j x_{ij} \right)
$$

La salida del regresor está en el intervalo $[0,1]$, podemos interpretarla como la probabilidad de "pertenecer" a la clase $1$ (o la probabilidad de "no pertenecer" a la clase 0)

> Modelo de clasificación binaria (dos clases)

### Ejemplo

Considera los siguientes datos sintéticos

In [ ]:
np.random.seed(1234)
data = np.concatenate((np.random.randn(50, 2), 
                       2 + np.random.randn(50, 2)), axis=0)
#data = data - np.mean(data, axis=0, keepdims=True)
label = np.array([0]*50 + [1]*50)

fig, ax = plt.subplots(1, figsize=(6, 3), tight_layout=True)
ax.scatter(data[label==0, 0], data[label==0, 1], c='k', s=20, label='clase 0')
ax.scatter(data[label==1, 0], data[label==1, 1], c='k', s=20, marker='x', label='clase 1')
plt.legend();

El siguiente ejemplo permite modificar los parámetros $\theta_0$, $\theta_1$ y $\theta_2$ de un regresor logístico

> ¿Puedes obtener un plano que separé las clases lo mejor posible?

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 3), tight_layout=True)

from matplotlib import cm
fig.colorbar(cm.ScalarMappable(cmap=plt.cm.RdBu_r), ax=ax)
x_min, x_max = data[:, 0].min() - 0.5, data[:, 0].max() + 0.5
y_min, y_max = data[:, 1].min() - 0.5, data[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

def sigmoid(X, w, b):
    Z = np.dot(X, w) + b # Transformación lineal
    return 1./(1 + np.exp(-Z)) # Función sigmoide

def update_plot(t1, t2, t0):    
    ax.cla()
    ax.contourf(xx, yy, sigmoid(np.c_[xx.ravel(), yy.ravel()], np.array([t1, t2]), t0).reshape(xx.shape), 
                cmap=plt.cm.RdBu_r, alpha=0.75)
    ax.scatter(data[label==0, 0], data[label==0, 1], c='k', s=20)
    ax.scatter(data[label==1, 0], data[label==1, 1], c='k', s=20, marker='x')
    
widgets.interact(update_plot, 
                 t1=FloatSlider_nice(value=-1, min=-10, max=10),
                 t2=FloatSlider_nice(value=1, min=-10, max=10),
                 t0=FloatSlider_nice(value=0, min=-10, max=10));

## Función de costo para clasificación binaria

La función de costo del regresor lineal no es apropiada ya que queremos comparar categorías discretas (clases)

Para un problema de clasificación binario podemos usar la **Entropía Cruzada Binaria**

$$
L(\theta) = \sum_{i=1}^N  -y_i \log( f_\theta(\vec x_i) ) - (1-y_i) \log(1 - f_\theta(\vec x_i))
$$

Calculemos el gradiente de la entropía cruzada

$$
\begin{align}
\frac{d}{d \theta_j} L(\theta) &= \sum_{i=1}^N  \left(-\frac{y_i}{f_\theta(\vec x_i)} + \frac{1-y_i}{1 - f_\theta(\vec x_i)}\right) \frac{d f_\theta(\vec x_i)}{d\theta_j} \nonumber \\
&= -\sum_{i=1}^N (y_i - f_\theta(\vec x_i)) \frac{d z_i}{d\theta_j} \nonumber \\
&= -\sum_{i=1}^N \left(y_i -  \mathcal{S} \left(z_i\right)  \right) x_{ij} \nonumber
\end{align}
$$

donde se uso que

$$
\frac{d \mathcal{S}(z)}{dz} = \mathcal{S}(z) (1- \mathcal{S}(z))
$$

Si observamos

$$
\frac{d}{d \theta_j} L(\theta) = -\sum_{i=1}^N \left(y_i -  \mathcal{S} \left(z_i\right)  \right) x_{ij}
$$

podemos notar que no es posible despejar analiticamente $\theta_j$ como en el caso de la regresión lineal. Esto se debe a la no-linealdad que introduce la función sigmoide

> Un opción es proponer una $\theta$ inicial y mejorarlo interativamente: Métodos de optimización iterativos

## Optimización: Método de Newton

Sea el valor actual del vector de parámetros $\theta_t$

Queremos encontrar el mejor "próximo valor" según nuestra función objetivo
$$
\theta_{t+1} = \theta_t + \Delta \theta
$$
Consideremos la aproximación de Taylor de segundo orden de $f$
$$
f(\theta_{t} + \Delta \theta) \approx f(\theta_t) + \nabla f (\theta_t) \Delta \theta + \frac{1}{2} \Delta \theta^T H_f (\theta_t) \Delta \theta 
$$
Derivando en función de $\Delta \theta$ e igualando a cero tenemos
$$
\begin{align}
\nabla f (\theta_t)  +  H_f (\theta_t) \Delta \theta &= 0 \nonumber \\
\Delta \theta &= - [H_f (\theta_t)]^{-1}\nabla f (\theta_t)  \nonumber \\
\theta_{t+1} &= \theta_{t} - [H_f (\theta_t)]^{-1}\nabla f (\theta_t)  \nonumber \\
\end{align}
$$

- Se obtiene una regla iterativa en función del **Gradiente** y del **Hessiano**
- La solución depende de $\theta_0$
- "Asumimos" que la aproximación de segundo orden es "buena"
- Si nuestro modelo tiene $M$ parámetros el Hessiano es de $M\times M$, ¿Qué pasa si $M$ es grande?



## Optimización: Gradiente descendente

Muchas veces el Hessiano es prohibitivo 

Podemos sacrificar calidad y reducir cómputo usando una aproximación de primer orden

El método más clásico es el **gradiente descendente**

$$
\theta_{t+1} = \theta_{t} - \eta \nabla f (\theta_t)
$$

donde hemos reemplazado el Hessiano por una constante $\eta$ llamado "paso" o "tasa de aprendizaje"

- ¿Cómo cambia la optimización con distintos $\eta$?
- ¿Qué ocurre cuando la superficie de error tiene mínimos locales?

### Ejercicio

- Considere la función $5 + (\theta -1)^2$, calcule su gradiente con respecto a $\theta$, complete la función `df` con su solución
- Repita para la función $5 + (\theta -1)^2 + 10\sin(\theta)$

In [ ]:
f = lambda theta : 5+ (theta-1.)**2 #+ 10*np.sin(theta)
df = 0 # Completa aquí!

- Estudie como se optimiza un conjunto de soluciones iniciales usando distintos valores de la tasa de aprendizaje $\eta$
- Considere las funciones de costo anterior por separado

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(7, 4), tight_layout=True, sharex=True)
x = np.linspace(-4, 6, num=100)

t = 10*np.random.rand(10) - 4. # valores iniciales de theta
ax[0].plot(x, f(x))
sc = ax[0].scatter(t, f(t), s=100)
ax[1].plot(x, -df(x))
ax[1].set_xlabel(r'$\theta$')
ax[0].set_ylabel(r'$f(\theta)$')
ax[1].set_ylabel(r'$-\nabla f(\theta)$')

eta = 0.01 # tasa de aprendizaje

def update(n):
    t = sc.get_offsets()[:, 0]
    t -= eta*df(t) # Gradiente descendente
    sc.set_offsets(np.c_[t, f(t)])
    
anim = animation.FuncAnimation(fig, update, frames=100, interval=200, repeat=False, blit=True)

## Gradiente descendente para el regresor logístico

Ajustemos el regresor del ejemplo anterior usando gradiente descedente

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 3), tight_layout=True)
fig.colorbar(cm.ScalarMappable(cmap=plt.cm.RdBu_r), ax=ax[0])

def sigmoid(X, theta):
    Z = np.dot(X, theta[1:]) + theta[0] # Transformación lineal
    return 1./(1 + np.exp(-Z)) # Función sigmoide

def loss(theta, X, Y, tol=1e-8): # Función de costo
    f = sigmoid(X, theta)
    return -np.sum(Y*np.log(f + tol) + (1.-Y)*np.log(1.-f+tol))

def grad_loss(theta, X, Y): # Gradiente
    f = sigmoid(X, theta)
    return -np.dot(Y-f, np.hstack((np.ones(shape=(len(data), 1)), data)))

eta = 1e-2 # Tasa de aprendizaje
np.random.seed(1234)
t = 0.5*np.random.randn(3) # Solución inicial
L_history = []

def update_plot(k):    
    global t, L_history    
    L_history.append(loss(t, data, label)) # Valor histórico de la loss
    
    gradL = grad_loss(t, data, label)
    t -= eta*gradL # Gradiente descendente
    
    ax[0].cla()
    ax[0].contourf(xx, yy, sigmoid(np.c_[xx.ravel(), yy.ravel()], t).reshape(xx.shape), 
                cmap=plt.cm.RdBu_r, alpha=0.75)
    ax[0].scatter(data[label==0, 0], data[label==0, 1], c='k', s=20)
    ax[0].scatter(data[label==1, 0], data[label==1, 1], c='k', s=20, marker='x')
    ax[1].cla()
    ax[1].plot(L_history)

anim = animation.FuncAnimation(fig, update_plot, frames=50, interval=250, repeat=False, blit=True)

# Métricas: Evaluando un clasificador binario

La salida de este clasificador es un valor en el rango $[0, 1]$

Para tomar un decisión binaria se debe seleccionar un umbral $\mathcal{T}$ tal que

$$
d_i = 
\begin{cases} 
0, & \text{si } f_\theta(\vec x_i)  < \mathcal{T} \\ 
1, & \text{si } f_\theta(\vec x_i) \geq \mathcal{T}
\end{cases}
$$

Una vez seleccionado el umbral se puede contar la cantidad de 
- **True positives** (TP): Era clase (1) y lo clasifico como (1)
- **True negative** (TN): Era clase (0) y lo clasifico como (0)
- **False positives** (FP): Era clase (0) y lo clasifico como (1): Error tipo I
- **False negative** (FN): Era clase (1) y lo clasifico como (0): Error tipo II

A partir de estas métricas se construye la **matriz de confusión** del clasificador

|Clasificado como/En realidad era|Positivo|Negativo|
|---|---|---|
|Positivo:|TP | FP |
|Negativo:| FN | TN |

En base a estas métricas se construyen otras 

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

también conocida como la **Tasa de verdaderos positivos** (TPR) o sensitividad

> TPR: La proporción de positivos correctamente clasificados respecto al total de positivos

$$
\text{FPR} = \frac{FP}{TN + FP} = 1 - \frac{TN}{TN + FP}
$$

la **tasa de falsos positivos** (FPR) también representada como "1 - especificidad"


> FPR: La proporción de negativos incorrectamente clasificados respecto al total de negativos

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

también conocido como pureza

> Precision: La proporción de positivos correctamente clasificados respecto a todos los ejemplos clasificados como positivo

$$
\text{Accuracy} = \frac{TP+TN}{TP + FP + FN+ TN}
$$

> Accuracy: La proporción de ejemplos correctamente clasificados

$$
\text{f1-score} = \frac{2\cdot\text{Recall}\cdot\text{Precision}}{\text{Recall} + \text{Precision}}
$$

> f1-score: Media armónica entre Recall y Precision asumiendo igual ponderación

Si las clases son desbalanceadas entonces f1-score es más aconsejable que accuracy

### Ejemplo: Evaluando el regresor logístico

Podemos usar las funciones de `sklearn.metrics` para obtener matrices de confusión, reportes y curvas de desempeño

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

probability = sigmoid(data, t)

display("Matriz de confusión:")
display(confusion_matrix(y_true=label, y_pred=probability>0.5))

display("Matriz de confusión:")
print(classification_report(y_true=label, y_pred=probability>0.5))

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

fpr, tpr, th = roc_curve(y_true=label, y_score=probability)

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
ax[0].plot(fpr, tpr);
ax[0].set_xlabel('Tasa de Falsos positivos')
ax[0].set_ylabel('Tasa de Verdaderos\nPositivos (Recall)')
ax[0].set_title(f'Curva ROC AUC: {auc(fpr, tpr)}')

prec, rec, th = precision_recall_curve(y_true=label, probas_pred=probability)

ax[1].plot(rec, prec, '-');
ax[1].set_xlabel('Recall')
ax[1].set_ylabel('Precision')
ax[1].set_title('Curva PR');

# Ojo con:


#### La distribución de los datos donde se aplicará el clasificador es distinta a la que usaste para entrenar/validar

Actualiza tus conjuntos de datos para que sean representativos!

#### Usa los subconjuntos adecuadamente

Ajusta los parámetros con el set de validación

Compara distintas familias de modelos con el set de prueba


#### La métrica que usas no es la adecuada para el problema

Si el problema tiene clases desbalanceadas el *accuracy* puede ser muy alto, contrasta usando métricas sencibles al desbalance (e.g. *f1-score*)

#### Mi modelo se sobreajusta de inmediato

Prueba disminuyendo la complejidad/arquitectura del modelo o añadiendo **regularización**

Esto también puede ser señal de que necesitas más ejemplos para entrenar

Se pueden usar **Técnicas de aumentación** de datos

#### Mi modelo no está aprendiendo

Si estas seguro que no hay bugs prueba aumentando la complejidad del modelo

#### Estudia los errores de tu modelo para mejorarlo

Analiza los datos mal clasificados y busca patrones

Revisa que las etiquetas estén correctas

Revisa que los atributos estén adecuadamente calculados

Propon nuevos atributos que ayuden a clasificador los ejemplos difíciles

